# 0.0: pull packages

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import joblib
import numpy as np
import random
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model, model_selection, ensemble, svm, naive_bayes, preprocessing
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, r2_score, mean_absolute_error, mean_squared_error
from sklearn.decomposition import FactorAnalysis
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import getpass
import pyodbc
import sys
import sqlalchemy
import urllib
from datetime import date
from collections import Counter
import xgboost as xgb
from xgboost.sklearn import XGBClassifier, XGBRegressor
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
import seaborn as sns
import time
from catboost import Pool, CatBoostClassifier, CatBoostRegressor

## 0.1: pull data

In [3]:
df_sql = pd.DataFrame() 
df_sql['servers'] = [serverlist]
df_sql['databases'] = [databaselist]

In [4]:
for a,b in zip(df_sql.servers, df_sql.databases): 
    if a == df_sql['servers'][0] and b == df_sql['databases'][0]:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
 
        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_001_ACA_Info')
        df_mem = pd.DataFrame(result.fetchall(),columns=result.keys())

    else:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_001_ACA_Info')
        df_mem = df_mem.append(pd.DataFrame(result.fetchall(),columns=result.keys()))
        df_mem['clientDB'] = df_mem['clientDB'].fillna(b)
        
    if a == df_sql['servers'][0] and b == df_sql['databases'][0]:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
 
        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_002_capturedconditions')
        df_cc = pd.DataFrame(result.fetchall(),columns=result.keys())
        df_cc['clientDB'] = b

    else:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_002_capturedconditions')
        df_cc = df_cc.append(pd.DataFrame(result.fetchall(),columns=result.keys()))
        df_cc['clientDB'] = df_cc['clientDB'].fillna(b)
        
    if a == df_sql['servers'][0] and b == df_sql['databases'][0]:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
 
        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_003_nonscoredconditions')
        df_noncc = pd.DataFrame(result.fetchall(),columns=result.keys())
        df_noncc['clientDB'] = b

    else:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_003_nonscoredconditions')
        df_noncc = df_noncc.append(pd.DataFrame(result.fetchall(),columns=result.keys()))
        df_noncc['clientDB'] = df_noncc['clientDB'].fillna(b)
        
    if a == df_sql['servers'][0] and b == df_sql['databases'][0]:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
 
        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_004_procedures')
        df_procs = pd.DataFrame(result.fetchall(),columns=result.keys())
        df_procs['clientDB'] = b

    else:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_004_procedures')
        df_procs = df_procs.append(pd.DataFrame(result.fetchall(),columns=result.keys()))
        df_procs['clientDB'] = df_procs['clientDB'].fillna(b)
        
    if a == df_sql['servers'][0] and b == df_sql['databases'][0]:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
 
        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_005_drugs')
        df_drugs = pd.DataFrame(result.fetchall(),columns=result.keys())
        df_drugs['clientDB'] = b

    else:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_005_drugs')
        df_drugs = df_drugs.append(pd.DataFrame(result.fetchall(),columns=result.keys()))
        df_drugs['clientDB'] = df_drugs['clientDB'].fillna(b)
        
    if a == df_sql['servers'][0] and b == df_sql['databases'][0]:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
 
        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_006_county_tpd')
        df_tpd = pd.DataFrame(result.fetchall(),columns=result.keys())

    else:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_006_county_tpd')
        df_tpd = df_tpd.append(pd.DataFrame(result.fetchall(),columns=result.keys()))
        df_tpd['clientDB'] = df_tpd['clientDB'].fillna(b)
        
    if a == df_sql['servers'][0] and b == df_sql['databases'][0]:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
 
        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_008_targets')
        df_targets = pd.DataFrame(result.fetchall(),columns=result.keys())

    else:
        params = urllib.parse.quote_plus("DSN={};DATABASE={};Trusted_Connection=yes".format(a,b))
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

        result = engine.execute('SELECT DISTINCT * FROM Y_DS_NewMember_008_targets')
        df_targets = df_targets.append(pd.DataFrame(result.fetchall(),columns=result.keys()))
        df_targets['clientDB'] = df_targets['clientDB'].fillna(b)

## 0.2: prepare data

In [5]:
df_cc_pivot = pd.pivot_table(df_cc, values='instances', index=['clientDSMemberKey','clientDB','active_year'], columns=['CC']).add_suffix('_captured')
df_noncc_pivot = pd.pivot_table(df_noncc, values='instances', index=['clientDSMemberKey','clientDB','active_year'], columns=['CC']).add_suffix('_noncaptured')
df_drugs['drug_subclass'] = pd.to_numeric(df_drugs.drug_subclass)
df_drugs100 = df_drugs[['drug_subclass','instances']].groupby('drug_subclass').sum().sort_values('instances',ascending=False)[0:100].reset_index()
df_drugs=df_drugs.merge(df_drugs100, how='inner', on='drug_subclass').drop('instances_y',axis=1)
df_drugs_pivot = pd.pivot_table(df_drugs, values='instances_x', index=['clientDSMemberKey','clientDB','active_year'],columns=['drug_subclass'])
df_drugs_pivot.columns = df_drugs_pivot.columns.astype(str)
df_procs['description'] = df_procs['category'] + "_" + df_procs['class']
df_procs_pivot = pd.pivot_table(df_procs, values='instances', index=['clientDSMemberKey','clientDB','active_year'],columns=['description'])
df_tpd_pivot = pd.pivot_table(df_tpd, values='value', index=['clientDSMemberKey','clientDB'],columns=['metric'])
df_noncc_pivot = df_noncc_pivot.reset_index()
df_cc_pivot = df_cc_pivot.reset_index()
df_drugs_pivot = df_drugs_pivot.reset_index()
df_procs_pivot = df_procs_pivot.reset_index()
df_tpd_pivot = df_tpd_pivot.reset_index()

In [6]:
df_final = pd.merge(df_mem, df_noncc_pivot, on=['clientDSMemberKey','clientDB','active_year'],how='left')
df_final = pd.merge(df_final, df_cc_pivot, on=['clientDSMemberKey','clientDB','active_year'],how='left')
df_final = pd.merge(df_final, df_drugs_pivot, on=['clientDSMemberKey','clientDB','active_year'],how='left')
df_final = pd.merge(df_final, df_procs_pivot, on=['clientDSMemberKey','clientDB','active_year'],how='left')
df_final = pd.merge(df_final, df_tpd_pivot, on=['clientDSMemberKey','clientDB'],how='left')

In [9]:
df_final['sex'] = df_final['sex'].map({'M':1, 'F':0})
df_final['relationship']=df_final['relationship'].map({' ':0})

In [10]:
del df_noncc_pivot
del df_cc_pivot
del df_drugs_pivot
del df_procs_pivot
del df_tpd_pivot
del df_noncc
del df_cc
del df_drugs
del df_procs
del df_tpd

In [11]:
df_final = pd.merge(df_final, df_targets, on=['clientDSMemberKey','clientDB','active_year'],how='inner')

## 0.3: prepare targets and features

In [12]:
df_final['sumCost'] = df_final['medClaimTotal_y']+df_final['rxClaimTotal_y']
df_final['drops'] = ((df_final['Coverage_Months_Actual'] <= 2) & (df_final['Coverage_Months_Actual'] > 1))*1

In [13]:
df_final['medClaimTotal_x']=df_final['medClaimTotal_x'].astype('float')
df_final['rxClaimTotal_x']=df_final['rxClaimTotal_x'].astype('float')
df_final['sumCost']=df_final['sumCost'].astype('float')

In [14]:
df_final['metalLevel']=df_final['metalLevel'].astype('int')

In [15]:
features = [featurelist]

# 1.0: cost model - catboost

In [16]:
df_final.loc[df_final['sumCost']/12 < 100, 'costCatActual'] = 'low'
df_final.loc[df_final['sumCost']/12 >= 2500, 'costCatActual'] = 'high'
df_final.fillna({'costCatActual':'med'}, inplace = True)

In [17]:
df_final['costCatActual'].value_counts()

low     1125918
med      481335
high      67031
Name: costCatActual, dtype: int64

In [19]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_final[features], df_final['costCatActual'], test_size=0.1,random_state=5)
categorical_features_indices = np.where(df_final[features].dtypes != np.float)[0]

In [20]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [21]:
def modelfit_classification(alg, X_train, y_train, X_test, y_test, categorical_features_indices, useTrainCV=True, cv_folds=3, early_stopping_rounds=50):
    
    
    #Fit the algorithm on the data
    alg.fit(X_train, y_train, categorical_features_indices)
        
    #Predict training set:
    dtrain_predictions = alg.predict(X_train)
    dtrain_predprob = alg.predict_proba(X_train)[:,1]
    
    #Predict test set:
    dtest_predictions = alg.predict(X_test)
    dtest_predprob = alg.predict_proba(X_test)[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(y_train, dtrain_predictions))
    print(classification_report(y_train, dtrain_predictions))
    print(confusion_matrix(y_train, dtrain_predictions, labels = ['low', 'med', 'high']))
    cnf_matrix = confusion_matrix(y_train, dtrain_predictions, labels = ['low', 'med', 'high'])
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=['low', 'med', 'high'],
                      title='Confusion matrix, without normalization')
    
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(y_test.values, dtest_predictions))
    print(classification_report(y_test, dtest_predictions))
    print(confusion_matrix(y_test, dtest_predictions, labels = ['low', 'med', 'high']))
    cnf_matrix = confusion_matrix(y_test, dtest_predictions, labels = ['low', 'med', 'high'])
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=['low', 'med', 'high'],
                      title='Confusion matrix, without normalization')
                    
    feat_imp = pd.DataFrame({'features':features})
    feat_imp['importance'] = alg.feature_importances_
    feat_imp = feat_imp.sort_values(by = 'importance', ascending=False).head(20)
    feat_imp.plot(x='features', y='importance', kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
catboost_class14 = CatBoostClassifier(
    iterations=250,
    learning_rate=0.1,
    depth=6,
    loss_function='MultiClassOneVsAll',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    thread_count = 4,
    l2_leaf_reg=1,
    random_state=27,
    class_weights = [1, 1, 1],
    ## high, low, med
    metric_period=10
)
modelfit_classification(catboost_class14, X_train, y_train, X_test, y_test, categorical_features_indices, useTrainCV=True, cv_folds=3, early_stopping_rounds=50)

In [ ]:
catboost_class15 = CatBoostClassifier(
    iterations=250,
    learning_rate=0.1,
    depth=6,
    loss_function='MultiClassOneVsAll',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    thread_count = 4,
    l2_leaf_reg=1,
    random_state=27,
    class_weights = [2, 1, 1],
    ## high, low, med
    metric_period=10
)
modelfit_classification(catboost_class15, X_train, y_train, X_test, y_test, categorical_features_indices, useTrainCV=True, cv_folds=3, early_stopping_rounds=50)

In [ ]:
catboost_class18 = CatBoostClassifier(
    iterations=500,
    learning_rate=0.2,
    depth=6,
    loss_function='MultiClassOneVsAll',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    thread_count = 4,
    l2_leaf_reg=1,
    random_state=27,
    class_weights = [2.5, 1, 1],
    ## high, low, med
    metric_period=10
)
modelfit_classification(catboost_class18, X_train, y_train, X_test, y_test, categorical_features_indices, useTrainCV=True, cv_folds=3, early_stopping_rounds=50)

# 1.1: very high cost testing

In [16]:
##very high testing
df_final.loc[df_final['sumCost']/12 < 100, 'costCatActual'] = 'low'
df_final.loc[(df_final['sumCost']/12 >= 2500) & (df_final['sumCost']/12 < 12000), 'costCatActual'] = 'high'
df_final.loc[df_final['sumCost']/12 >= 12000, 'costCatActual'] = 'very high'
df_final.fillna({'costCatActual':'med'}, inplace = True)

In [17]:
df_final['costCatActual'].value_counts()

low          1125918
med           481335
high           53011
very high      14020
Name: costCatActual, dtype: int64

In [18]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_final[features], df_final['costCatActual'], test_size=0.1,random_state=5)
categorical_features_indices = np.where(df_final[features].dtypes != np.float)[0]

In [19]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [20]:
def modelfit_classification(alg, X_train, y_train, X_test, y_test, categorical_features_indices, useTrainCV=True, cv_folds=3, early_stopping_rounds=50):
    
    
    #Fit the algorithm on the data
    alg.fit(X_train, y_train, categorical_features_indices)
        
    #Predict training set:
    dtrain_predictions = alg.predict(X_train)
    dtrain_predprob = alg.predict_proba(X_train)[:,1]
    
    #Predict test set:
    dtest_predictions = alg.predict(X_test)
    dtest_predprob = alg.predict_proba(X_test)[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(y_train, dtrain_predictions))
    print(classification_report(y_train, dtrain_predictions))
    print(confusion_matrix(y_train, dtrain_predictions, labels = ['low', 'med', 'high']))
    cnf_matrix = confusion_matrix(y_train, dtrain_predictions, labels = ['low', 'med', 'high'])
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=['low', 'med', 'high'],
                      title='Confusion matrix, without normalization')
    
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(y_test.values, dtest_predictions))
    print(classification_report(y_test, dtest_predictions))
    print(confusion_matrix(y_test, dtest_predictions, labels = ['low', 'med', 'high']))
    cnf_matrix = confusion_matrix(y_test, dtest_predictions, labels = ['low', 'med', 'high'])
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=['low', 'med', 'high'],
                      title='Confusion matrix, without normalization')
                    
    feat_imp = pd.DataFrame({'features':features})
    feat_imp['importance'] = alg.feature_importances_
    feat_imp = feat_imp.sort_values(by = 'importance', ascending=False).head(20)
    feat_imp.plot(x='features', y='importance', kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
catboost_class1 = CatBoostClassifier(
    iterations=250,
    learning_rate=0.1,
    depth=6,
    loss_function='MultiClassOneVsAll',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    thread_count = 4,
    l2_leaf_reg=1,
    random_state=27,
    class_weights = [1, 1, 1, 1],
    ##low, med, high, veryhigh
    metric_period=10
)
modelfit_classification(catboost_class1, X_train, y_train, X_test, y_test, categorical_features_indices, useTrainCV=True, cv_folds=3, early_stopping_rounds=50)

In [ ]:
catboost_class4 = CatBoostClassifier(
    iterations=250,
    learning_rate=0.1,
    depth=6,
    loss_function='MultiClassOneVsAll',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    thread_count = 4,
    l2_leaf_reg=1,
    random_state=27,
    class_weights = [1, 1, 2, 5],
    ##low, med, high, veryhigh
    metric_period=10
)
modelfit_classification(catboost_class4, X_train, y_train, X_test, y_test, categorical_features_indices, useTrainCV=True, cv_folds=3, early_stopping_rounds=50)

# 2.0: risk model - catboost

In [27]:
df_final.loc[df_final['TotalRiskTarget'] < 1, 'riskCatActual'] = 'low'
df_final.loc[df_final['TotalRiskTarget'] >= 3, 'riskCatActual'] = 'high'
df_final.fillna({'riskCatActual':'med'}, inplace = True)

In [28]:
df_final['riskCatActual'].value_counts()

low     1238194
med      213494
high     197843
Name: riskCatActual, dtype: int64

In [29]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_final[features], df_final['riskCatActual'], test_size=0.1,random_state=5)
categorical_features_indices = np.where(df_final[features].dtypes != np.float)[0]

In [30]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [31]:
def modelfit_classification(alg, X_train, y_train, X_test, y_test, categorical_features_indices, useTrainCV=True, cv_folds=3, early_stopping_rounds=50):
    
    
    #Fit the algorithm on the data
    alg.fit(X_train, y_train, categorical_features_indices)
        
    #Predict training set:
    dtrain_predictions = alg.predict(X_train)
    dtrain_predprob = alg.predict_proba(X_train)[:,1]
    
    #Predict test set:
    dtest_predictions = alg.predict(X_test)
    dtest_predprob = alg.predict_proba(X_test)[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(y_train, dtrain_predictions))
    print(classification_report(y_train, dtrain_predictions))
    print(confusion_matrix(y_train, dtrain_predictions, labels = ['low', 'med', 'high']))
    cnf_matrix = confusion_matrix(y_train, dtrain_predictions, labels = ['low', 'med', 'high'])
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=['low', 'med', 'high'],
                      title='Confusion matrix, without normalization')
    
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(y_test.values, dtest_predictions))
    print(classification_report(y_test, dtest_predictions))
    print(confusion_matrix(y_test, dtest_predictions, labels = ['low', 'med', 'high']))
    cnf_matrix = confusion_matrix(y_test, dtest_predictions, labels = ['low', 'med', 'high'])
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=['low', 'med', 'high'],
                      title='Confusion matrix, without normalization')
                    
    feat_imp = pd.DataFrame({'features':features})
    feat_imp['importance'] = alg.feature_importances_
    feat_imp = feat_imp.sort_values(by = 'importance', ascending=False).head(20)
    feat_imp.plot(x='features', y='importance', kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
catclass_risk21 = CatBoostClassifier(
    iterations=250,
    learning_rate=0.1,
    random_strength=0.1,
    depth=10,
    loss_function='MultiClassOneVsAll',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    thread_count = 4,
    class_weights = [2, 1, 2],
    ## high, low, med
    metric_period=10
)
modelfit_classification(catclass_risk21, X_train, y_train, X_test, y_test, categorical_features_indices, useTrainCV=True, cv_folds=3, early_stopping_rounds=50)

In [ ]:
catclass_risk22 = CatBoostClassifier(
    iterations=250,
    learning_rate=0.1,
    random_strength=0.1,
    depth=10,
    loss_function='MultiClassOneVsAll',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    thread_count = 4,
    class_weights = [1, 1, 1],
    ## high, low, med
    metric_period=10
)
modelfit_classification(catclass_risk22, X_train, y_train, X_test, y_test, categorical_features_indices, useTrainCV=True, cv_folds=3, early_stopping_rounds=50)

In [ ]:
catclass_risk25 = CatBoostClassifier(
    iterations=250,
    learning_rate=0.1,
    random_strength=0.1,
    depth=8,
    loss_function='MultiClassOneVsAll',
    eval_metric='Accuracy',
    leaf_estimation_method='Newton',
    thread_count = 4,
    class_weights = [3, 1, 1],
    ## high, low, med
    metric_period=10
)
modelfit_classification(catclass_risk25, X_train, y_train, X_test, y_test, categorical_features_indices, useTrainCV=True, cv_folds=3, early_stopping_rounds=50)

# 3.0: drop model - xgboost

In [16]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_final[features], df_final['drops'], test_size=0.1,random_state=5)

In [ ]:
len(df_final['drops'])/sum(df_final['drops'])

In [19]:
def modelfit_classification(alg, X_train, y_train, X_test, y_test, useTrainCV=True, cv_folds=3, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(X_train, y_train, eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(X_train)
    dtrain_predprob = alg.predict_proba(X_train)[:,1]
    
    #Predict test set:
    dtest_predictions = alg.predict(X_test)
    dtest_predprob = alg.predict_proba(X_test)[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(y_train, dtrain_predictions))
    print("AUC Score (Train): %f" % roc_auc_score(y_train, dtrain_predprob))
    print(classification_report(y_train, dtrain_predictions))
    print(confusion_matrix(y_train, dtrain_predictions))
    
    print("\nModel Report")
    print("Accuracy : %.4g" % accuracy_score(y_test.values, dtest_predictions))
    print("AUC Score (Train): %f" % roc_auc_score(y_test, dtest_predprob))
    print(classification_report(y_test, dtest_predictions))
    print(confusion_matrix(y_test, dtest_predictions))
                    
    feat_imp = pd.DataFrame({'features':features})
    feat_imp['importance'] = alg.feature_importances_
    feat_imp = feat_imp.sort_values(by = 'importance', ascending=False).head(20)
    feat_imp.plot(x='features', y='importance', kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
%%time
xgb5 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=50,
 n_jobs=4,
 max_depth=8,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 scale_pos_weight=15,
 max_delta_step=8,
 reg_alpha=100,
 seed=27)
modelfit_classification(xgb5, X_train, y_train, X_test, y_test, useTrainCV=False, cv_folds=3, early_stopping_rounds=50)

In [ ]:
%%time
xgb7 = XGBClassifier(
 learning_rate =0.2,
 n_estimators=50,
 n_jobs=4,
 max_depth=10,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 scale_pos_weight=12,
 max_delta_step=8,
 reg_alpha=100,
 seed=27)
modelfit_classification(xgb7, X_train, y_train, X_test, y_test, useTrainCV=False, cv_folds=3, early_stopping_rounds=50)